-- write a script that does the following actions:

1/ load csv ____.csv DONE

2/ clean data best you think DONE

3/ write a new csv with a similar name with the cleaned data DONE 

4/ output the following answers: DONE
	
    a. what is the distance (in meters) between the cheapest sale and the most recent sale?
	b. what is the median street number, in multi-family houses, sold between 05/11/1933 and 03/12/1998, in Sacramento?
	c. what is the city name, and its 3 most common zip codes, that has the 2nd highest amount of beds sold? 


In [37]:
import pandas as pd
import numpy as np
import re

In [523]:
real_state = pd.read_csv('/Users/joaosantos/Downloads/realestate_fraugster_case.csv',
                         sep=';',
                         index_col='sale_date'
                        )

In [531]:
real_state.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 985 entries, 1943-01-09 11:56:01 to 1934-06-05 04:16:37
Data columns (total 11 columns):
street       985 non-null object
city         985 non-null object
zip          985 non-null object
state        985 non-null object
beds         985 non-null float64
baths        985 non-null float64
sq__ft       985 non-null float64
type         985 non-null object
price        985 non-null object
latitude     985 non-null object
longitude    985 non-null object
dtypes: float64(3), object(8)
memory usage: 92.3+ KB


In [525]:
#dropping NAs
real_state.dropna(inplace=True)

In [527]:
#uniform naming of Cities and streets
real_state.loc[:,'city'] = real_state.loc[:,'city'].str.title()
real_state.loc[:,'street'] = real_state.loc[:,'street'].str.title()

In [354]:
#I used these functions to assess the presence of non fitting characters
real_state[~real_state['zip'].str.isnumeric()]
real_state[~real_state['price'].str.isnumeric()]
[re.findall(r'[^0-9\.\- ]', string) for string in real_state['latitude']]
[re.findall(r'[^0-9\.\-]', string) for string in real_state['longitude']]

In [108]:
#as well as value counts to check if there's any weird value or typo
real_state['city'].value_counts()
real_state['zip'].value_counts()
real_state['state'].value_counts()
real_state['type'].value_counts()
real_state['latitude'].value_counts()

In [528]:
#and then removed non alphabetic characters from str columns
real_state['city'] = [re.sub(pattern=r'[^a-zA-Z ]',string=string, repl='') for string in real_state['city']]
real_state['state'] = [re.sub(pattern=r'[^a-zA-Z]',string=string, repl='') for string in real_state['state']]
real_state['street'] = [re.sub(pattern=r'[^a-zA-Z0-9 ]',string=string, repl='') for string in real_state['street']]
real_state['type'] = [re.sub(pattern=r'[^a-zA-Z ]',string=string, repl='') for string in real_state['type']]

In [529]:
#and non numeric characters from numeric columns as well
real_state['price'] = [re.sub(pattern=r'[^0-9]',string=string, repl='') for string in real_state['price']]
real_state['zip'] = [re.sub(pattern=r'[^0-9]',string=string, repl='') for string in real_state['zip']]
real_state['latitude'] = [re.sub(pattern=r'[^0-9\.\-]',string=string, repl='') for string in real_state['latitude']]
real_state['longitude'] = [re.sub(pattern=r'[^0-9\.\-]',string=string, repl='') for string in real_state['longitude']]

In [345]:
#alternative for checking a more specific latitude/longitude pattern and removing extra characters
latitude_list = [re.split(pattern=r"^(\-?[0-9]+\.[0-9]+)",string=string) for string in real_state['latitude']]
latitude = []
for row in latitude_list:
    counter = 0
    for cell in row:
        if re.search(r"^\-?[0-9]+\.[0-9]+$", cell) != None:
            latitude.append(cell)
            counter += 1
    if counter == 0:
        latitude.append('')
    elif counter > 1:
        raise 'Possible duplication'


In [530]:
#clean index and convert it to DatetimeIndex
real_state.index = pd.to_datetime([re.sub(pattern=r'[^\:\-0-9 ]',string=string, repl='') for string in real_state.index.values])


In [358]:
#transform index to DatetimeIndex
real_state.index = pd.to_datetime(real_state.index)

In [532]:
#load cleaned data into CSV
real_state.to_csv('real_state_clean.csv')

In [533]:
real_state.head()

,street,city,zip,state,beds,baths,sq__ft,type,price,latitude,longitude
1943-01-09 11:56:01,3526 High St,Sacramento,95838,CA,2.0,1.0,836.0,Residential,59222,38.631913,-121.434879
1996-11-08 23:09:38,51 Omaha Ct,Sacramento,95823,CA,3.0,1.0,1167.0,Residential,68212,38.478902,-121.431028
1915-01-05 07:31:45,2796 Branch St,Sacramento,95815,CA,2.0,1.0,796.0,Residential,68880,38.618305,-121.443839
1998-10-22 04:46:05,2805 Janette Way,Sacramento,95815,CA,2.0,1.0,852.0,Residential,69307,38.616835,-121.439146
1972-01-05 20:52:32,6001 Mcmahon Dr,Sacramento,95824,CA,2.0,1.0,797.0,Residential,81900,38.51947,-121.435768


### a. what is the distance (in meters) between the cheapest sale and the most recent sale?

In [534]:
#get the cheapest sale and most recent sale latitude and longitude
cheapest_sale = real_state[['latitude','longitude']][real_state['price'] == real_state['price'].min()]
recent_sale = real_state[['latitude','longitude']].sort_index(axis=0, ascending=False).iloc[0,:]

In [535]:
#define function to output distance between two points in Kilometers using Haversine formula
# https://en.wikipedia.org/wiki/Haversine_formula
from math import cos, asin, sqrt
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295     #Pi/180
    a = 0.5 - cos((lat2 - lat1) * p)/2 + cos(lat1 * p) * cos(lat2 * p) * (1 - cos((lon2 - lon1) * p)) / 2
    return 12742 * asin(sqrt(a)) #2*R*asin...

In [536]:
#since there were 3 entries tied for cheapest sales, the distance between those 3 and the cheapest sale was calculated
cheapest_sale['distance'] = cheapest_sale.apply(lambda x: str(round(distance(pd.to_numeric(recent_sale[0]), 
                                                                             pd.to_numeric(recent_sale[1]),
                                                                             pd.to_numeric(x['latitude']),
                                                                             pd.to_numeric(x['longitude']))*1000)
                                                             )+'m', 
                                                axis=1)


In [537]:
distance_between_sales = cheapest_sale

In [538]:
distance_between_sales

,latitude,longitude,distance
1988-07-28 09:27:38,38.423251,-121.444489,53034m
1944-01-09 17:10:33,38.617718,-121.440089,32415m
1905-01-11 12:42:46,38.544627,-121.35796,38269m


### b. what is the median street number, in multi-family houses, sold between 05/11/1933 and 03/12/1998 , in Sacramento?

In [539]:
#first filter DF for conditions
filtered_data = real_state[(real_state['type']=='MultiFamily') & (real_state.index >= '05/11/1933') 
           & (real_state.index <= '03/12/1998') & (real_state['city'] == 'Sacramento')]['street'].tolist()

#then split the numbers from the street string and put them in a clean list, already as int to be used to calculate median
street_numbers = []
for row in filtered_data:
    street_numbers.append(int(list(filter(None,re.split(pattern=r"[^0-9]",string=row)))[0]))
    


In [540]:
#calculate median from list of street numbers
np.median(street_numbers)

2820.0

### c. what is the city name, and its 3 most common zip codes, that has the 2nd highest amount of beds sold?

In [541]:
#get first the city with the 2nd highest amount in beds sold and use it as a mask to narrow it down to that particular 
#city and then perform a value counts on that DF to get the top3 zip codes
real_state[real_state['city']==real_state.groupby('city')[['beds','city']].sum().sort_values('beds').index[-2]
          ]['zip'].value_counts()

95758    44
95757    36
95624    34
Name: zip, dtype: int64